In [1]:
#Import all packages needed
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt #Visualisierungs Package (Vorlesung 5)
#Magic command, damit die Diagramme im Notebook angezeigt werden
%matplotlib inline 
import json  #Package um mit json files arbeiten zu können
import seaborn as sns #Visualisierungs Package (Vorlesung 5)

# Aufgabe

Quelle: *Diese Übung ist ein Teil aus einem Cornerstone Projekt aus dem Data Analyst Nanodegree von Udacity. Für diese Übung wurden einige Vereinfachungen vorgenommen.*

**Beschreibung**:

Habt Ihr jemals von dem Twitter-Account WeRateDogs gehört? Die Leute im Internet sind verrückt nach diesem Twitter-Account. Ich möchte Euch heute zeigen, warum. 

Heute hat WeRateDogs über 8,8 Millionen Follower. Das Schlüsselkonzept besteht darin, ein hübsches Bild der Doggos, Floofer, Pupper oder Puppos (so definieren sie das Entwicklungsstadium der Hunde) mit einer coolen Bildunterschrift und einer Bewertung mit einem Nenner von 10, aber einem Zähler, der alles sein kann, zu haben. In den meisten Fällen geht der Zähler deutlich über 10.


<div>
<img src="src/Picture 1.jpg" width="400"/>
</div>

**Inputdaten:**

Ingesamt haben wir 3 verschiedene Dateien, in denen sich aber leider verschiedene Fehler in den Daten befinden
- Das Twitter Archive der Tweets
- Eine Datei, in der die Rasse des Hundes vorausgesagt wurde
- Eine Datei, in der die Anzahl der Likes, Retweets etc pro Tweet steht


**Ziel**: 

Die Dateien wie unten vorgeschrieben zu bereinigen und zu einem Dataframe zusammenzufassen.

### Dokumentation der einzelnen Spalten der verschiedenen Quellen

Dokumentation Spalten ***Twitter_archive Dataframe***

Dieses Dataframe zeigt die Meta Daten jedes einzelnen Tweets für den betrachteten Zeitraum an.

- ***tweet_id*** ist die eineindeutige Twitter Post ID
- ***in_reply_to_status_id*** nicht relevant für uns
- ***in_reply_to_user_id*** nicht relevant für uns
- ***timestamp*** Zeitpunkt des posts
- ***source*** nicht relevant für uns
- ***text*** Tweet text
- ***retweeted_status_id*** ID wenn es ein retweet ist
- ***retweeted_status_user_id*** User ID wenn es ein retweet ist
- ***retweeted_status_user_timestamp*** User ID wenn es ein retweet ist
- ***expanded_urls*** URL des Tweets
- ***rating_numerator*** Zähler der Bewertung
- ***rating_denominator*** Nenner der Bewertung
- ***name*** Name des Hundes
- ***doggo*** Hundeentwicklungsstatus
- ***floofer*** Hundeentwicklungsstatus
- ***pupper*** Hundeentwicklungsstatus
- ***puppo*** Hundeentwicklungsstatus

### Dokumentation Spalten ***Image_prediction Dataframe***

Dieses DataFrame stammt aus einem Prediction Maschine Learning Algorithmus, welches an Hand der Bilder, versucht vorauszusagen, welche Rasse der Hund ist

- ***tweet_id*** ist die eineindeutige Twitter Post ID
- ***jpg_url*** URL des Bildes / der Bilder
- ***img_num*** Anzahl der Bilder
- ***p1/p2/p3*** Predicted Hunderasse
- ***p1_conf/p2_conf/p3_conf*** Wert aus der Klassifikation
- ***p1_dog/p2_dog/p3_dog*** Boolean ob der vorausgesagte p1/p2/p3 Wert ein Hund ist ja oder nein



### Dokumentation Spalten ***Tweet_json Dataframe***

Dieses DataFrame zeigt die numerischen Werte der einzelnen Tweets an.

- ***tweet_id*** ist die eineindeutige Twitter Post ID
- ***favorite_count*** Wie viele Favorites der Tweet bekommen hat
- ***retweet_count*** Wie oft der Tweet geretweeted wurde
- ***followers_count*** wie viele Follower es damals gab
- ***retweeted_status*** Ob der Tweet der Original Tweet war oder ein Retweet
- ***url*** URL des Tweets


## Gathering Data

Die nachfolgenden Zellen bis zum Start Der Übung dienen lediglich für Informationszwecke und werden vorgegeben, damit ihr euch schneller auf den wichtigten Teil der Übung, das Data Cleaning fokussieren könnt.

In [2]:
#Twitter Archive (Dateiformat CSV) in einen DataFrame laden
twitter_archive = pd.read_csv("src/twitter-archive-enhanced.csv")

In [3]:
#Bild-Vorhersagen (Image-Predictions - TSV file) in einen DataFrame laden
image_prediction = pd.read_csv('src/image-predictions.tsv', sep='\t' )

In [4]:
'''
Herauslesen der interessanten Informationen aus JSON-Wörterbüchern in einer Txt-Datei
und in einen DataFrame tweets_json einfügen
'''

my_demo_list = []
with open('src/tweet_json.txt', encoding='utf-8') as json_file:  
    all_data = json.load(json_file)
    for each_dictionary in all_data:
        tweet_id = each_dictionary['id']
        whole_tweet = each_dictionary['text']
        only_url = whole_tweet[whole_tweet.find('https'):]
        favorite_count = each_dictionary['favorite_count']
        retweet_count = each_dictionary['retweet_count']
        followers_count = each_dictionary['user']['followers_count']
        retweeted_status = each_dictionary['retweeted_status'] = each_dictionary.get('retweeted_status', 'Original tweet')
        if retweeted_status == 'Original tweet':
            url = only_url
        else:
            retweeted_status = 'This is a retweet'
            url = 'This is a retweet'
            
        my_demo_list.append({'tweet_id': str(tweet_id),
                             'favorite_count': int(favorite_count),
                             'retweet_count': int(retweet_count),
                             'followers_count': int(followers_count),
                             'retweeted_status': retweeted_status,
                            })
        tweet_json = pd.DataFrame(my_demo_list, columns = ['tweet_id', 'favorite_count','retweet_count', 
                                                           'followers_count', 'retweeted_status', 'url'])
                             
                            

## Assess

In [5]:
twitter_archive.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   tweet_id                    2356 non-null   int64  
 1   in_reply_to_status_id       78 non-null     float64
 2   in_reply_to_user_id         78 non-null     float64
 3   timestamp                   2356 non-null   object 
 4   source                      2356 non-null   object 
 5   text                        2356 non-null   object 
 6   retweeted_status_id         181 non-null    float64
 7   retweeted_status_user_id    181 non-null    float64
 8   retweeted_status_timestamp  181 non-null    object 
 9   expanded_urls               2297 non-null   object 
 10  rating_numerator            2356 non-null   int64  
 11  rating_denominator          2356 non-null   int64  
 12  name                        2356 non-null   object 
 13  doggo                       2356 

In [6]:
### FINDINGS ###
# Es gibt sowohl Originalbewertungen und Retweets
# Zeitstempel ist in einem nicht korrekten Datenformat (hier Datetime)
# Fehlende Daten in den folgenden Spalten: in_reply_to_status_id, in_reply_to_user_id, retweeted_status_id, retweeted_status_user_id, retweeted_status_timestamp, expanded_urls
# Hundeentwicklungsstufen sind in 4 Spalten

In [7]:
twitter_archive.sample(5)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
826,769335591808995329,NaN,NaN,2016-08-27 00:47:53 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: Ever seen a dog pet another dog...,7.069045e+17,4.196984e+09,2016-03-07 18:09:06 +0000,"https://vine.co/v/iXQAm5Lrgrh,https://vine.co/...",13,10,None,None,None,None,None
854,765222098633691136,NaN,NaN,2016-08-15 16:22:20 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Gromit. He's pupset because there's no...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/765222098...,10,10,Gromit,None,None,None,None
331,832998151111966721,NaN,NaN,2017-02-18 17:00:10 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Rhino. He arrived at a shelter with an...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/832998151...,13,10,Rhino,doggo,None,None,None
912,757596066325864448,NaN,NaN,2016-07-25 15:19:12 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here's another picture without a dog in it. Id...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/757596066...,4,10,None,None,None,None,None
796,773247561583001600,NaN,NaN,2016-09-06 19:52:39 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Chip. He's a pupholder. Comes with the...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/773247561...,10,10,Chip,None,None,None,None


In [8]:
### FINDINGS ###
# Hundestufen sind nicht korrekt angegeben
# Der Nenner der Bewertung ist zum Teil größer als 10

In [9]:
sum(twitter_archive.name.unique()=='None')

1

In [10]:
### FINDINGS ###
# Manchmal ist der Name des Hundes None

In [11]:
image_prediction.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   tweet_id  2075 non-null   int64  
 1   jpg_url   2075 non-null   object 
 2   img_num   2075 non-null   int64  
 3   p1        2075 non-null   object 
 4   p1_conf   2075 non-null   float64
 5   p1_dog    2075 non-null   bool   
 6   p2        2075 non-null   object 
 7   p2_conf   2075 non-null   float64
 8   p2_dog    2075 non-null   bool   
 9   p3        2075 non-null   object 
 10  p3_conf   2075 non-null   float64
 11  p3_dog    2075 non-null   bool   
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [12]:
image_prediction.sample(20)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
1432,773308824254029826,https://pbs.twimg.com/media/CrtYRMEWIAAUkCl.jpg,1,shopping_cart,0.572349,False,Labrador_retriever,0.151406,True,shopping_basket,0.107102,False
556,677673981332312066,https://pbs.twimg.com/media/CWeU5LBWEAA8F0J.jpg,1,Maltese_dog,0.817908,True,Angora,0.077805,False,Pomeranian,0.022184,True
807,692017291282812928,https://pbs.twimg.com/media/CZqKDZTVIAEvtbc.jpg,1,Tibetan_terrier,0.247565,True,cocker_spaniel,0.121377,True,bow_tie,0.099363,False
1960,866334964761202691,https://pbs.twimg.com/media/DAXXDQNXgAAoYQH.jpg,1,Samoyed,0.984086,True,Pomeranian,0.007919,True,keeshond,0.003328,True
1231,745789745784041472,https://pbs.twimg.com/media/ClmT0KHWkAAXbhy.jpg,1,Pekinese,0.984267,True,Shih-Tzu,0.008942,True,cocker_spaniel,0.001928,True
355,672594978741354496,https://pbs.twimg.com/media/CVWJkJXWsAInlZl.jpg,1,Great_Pyrenees,0.755945,True,Old_English_sheepdog,0.082337,True,Afghan_hound,0.027037,True
526,676613908052996102,https://pbs.twimg.com/media/CWPQwmJWUAAu_At.jpg,1,book_jacket,0.493790,False,Doberman,0.096423,True,miniature_pinscher,0.070647,True
1668,813051746834595840,https://pbs.twimg.com/media/C0iKPZIXUAAbDYV.jpg,1,golden_retriever,0.914804,True,Labrador_retriever,0.083550,True,kuvasz,0.000453,True
627,680836378243002368,https://pbs.twimg.com/media/CXLREjOW8AElfk6.jpg,3,Pembroke,0.427781,True,Shetland_sheepdog,0.160669,True,Pomeranian,0.111250,True
1737,821886076407029760,https://pbs.twimg.com/media/C2ftAxnWIAEUdAR.jpg,1,golden_retriever,0.266238,True,cocker_spaniel,0.223325,True,Irish_setter,0.151631,True


In [13]:
### FINDINGS ###
# p1, p2, p3 Hunderassen sind manchmal groß, manchmal klein geschrieben
# Die Spalten p1, p2 und p3 enthalten ungültige Daten, wie z. B. ein Vogelhaus, einen Dosenöffner oder einen Brustpanzer usw.

In [14]:
tweet_json.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2331 entries, 0 to 2330
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   tweet_id          2331 non-null   object 
 1   favorite_count    2331 non-null   int64  
 2   retweet_count     2331 non-null   int64  
 3   followers_count   2331 non-null   int64  
 4   retweeted_status  2331 non-null   object 
 5   url               0 non-null      float64
dtypes: float64(1), int64(3), object(2)
memory usage: 109.4+ KB


In [15]:
### FINDINGS ###
# tweet_id ist ein String und keine Zahl

In [16]:
tweet_json.sample(10)

,tweet_id,favorite_count,retweet_count,followers_count,retweeted_status,url
1942,673342308415348736,1212,544,8808643,Original tweet,NaN
1354,701570477911896070,2778,914,8808535,Original tweet,NaN
1336,703268521220972544,1956,544,8808535,Original tweet,NaN
641,791406955684368384,13289,4166,8808457,Original tweet,NaN
259,841077006473256960,22614,5232,8808447,Original tweet,NaN
2068,670780561024270336,757,273,8808646,Original tweet,NaN
464,815736392542261248,9927,2279,8808455,Original tweet,NaN
1557,687124485711986689,2152,499,8808541,Original tweet,NaN
844,762035686371364864,31838,15522,8808463,Original tweet,NaN
1680,680889648562991104,1739,357,8808546,Original tweet,NaN


In [17]:
tweet_json.retweeted_status.unique()

array(['Original tweet', 'This is a retweet'], dtype=object)

In [18]:
### FINDINGS ###
# Retweets in diesem df

In [19]:
# Doppelte Bilder in den URLs enthalen?
image_prediction.jpg_url.value_counts()

https://pbs.twimg.com/media/CZhn-QAWwAASQan.jpg                                            2
https://pbs.twimg.com/media/Cq9guJ5WgAADfpF.jpg                                            2
https://pbs.twimg.com/ext_tw_video_thumb/807106774843039744/pu/img/8XZg1xW35Xp2J6JW.jpg    2
https://pbs.twimg.com/media/CU1zsMSUAAAS0qW.jpg                                            2
https://pbs.twimg.com/media/CsrjryzWgAAZY00.jpg                                            2
                                                                                          ..
https://pbs.twimg.com/media/CXrmMSpUwAAdeRj.jpg                                            1
https://pbs.twimg.com/media/CXrawAhWkAAWSxC.jpg                                            1
https://pbs.twimg.com/media/CXrIntsUsAEkv0d.jpg                                            1
https://pbs.twimg.com/media/CXqcOHCUQAAugTB.jpg                                            1
https://pbs.twimg.com/media/DGKD1-bXoAAIAUK.jpg                       

In [20]:
#Anzahl doppelter URLS
len(image_prediction.jpg_url) - image_prediction.jpg_url.nunique()

66

In [21]:
### FINDINGS ###
# Es gibt doppelte Werte (an Hand der URL) 

In [22]:
twitter_archive.rating_numerator.value_counts()

12      558
11      464
10      461
13      351
9       158
8       102
7        55
14       54
5        37
6        32
3        19
4        17
2         9
1         9
75        2
15        2
420       2
0         2
80        1
144       1
17        1
26        1
20        1
121       1
143       1
44        1
60        1
45        1
50        1
99        1
204       1
1776      1
165       1
666       1
27        1
182       1
24        1
960       1
84        1
88        1
Name: rating_numerator, dtype: int64

In [23]:
with pd.option_context('max_colwidth', 300):
    display(twitter_archive[twitter_archive['text'].str.contains(r"(\d+\.\d*\/\d+)")]
            [['tweet_id', 'text', 'rating_numerator', 'rating_denominator']])

/var/folders/mq/545q4wvs7qv1ykzg9rcc02380000gn/T/ipykernel_29053/311254286.py:2: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  display(twitter_archive[twitter_archive['text'].str.contains(r"(\d+\.\d*\/\d+)")]


,tweet_id,text,rating_numerator,rating_denominator
45,883482846933004288,"This is Bella. She hopes her smile made you smile. If not, she is also offering you her favorite monkey. 13.5/10 https://t.co/qjrljjt948",5,10
340,832215909146226688,"RT @dog_rates: This is Logan, the Chow who lived. He solemnly swears he's up to lots of good. H*ckin magical af 9.75/10 https://t.co/yBO5wu…",75,10
695,786709082849828864,"This is Logan, the Chow who lived. He solemnly swears he's up to lots of good. H*ckin magical af 9.75/10 https://t.co/yBO5wuqaPS",75,10
763,778027034220126208,This is Sophie. She's a Jubilant Bush Pupper. Super h*ckin rare. Appears at random just to smile at the locals. 11.27/10 would smile back https://t.co/QFaUiIHxHq,27,10
1689,681340665377193984,I've been told there's a slight possibility he's checking his mirror. We'll bump to 9.5/10. Still a menace,5,10
1712,680494726643068929,Here we have uncovered an entire battalion of holiday puppers. Average of 11.26/10 https://t.co/eNm2S6p9BD,26,10


In [24]:
### FINDINGS ###
# Wenn der Zähler eine Dezimalzahl ist wie z.B. 13.5 steht in der Spalte rating_numerator nur die ".5" 

![purple-divider](https://user-images.githubusercontent.com/7065401/52071927-c1cd7100-2562-11e9-908a-dde91ba14e59.png)


# Start der Übung

## Zusammenfassung der Beobachtungen aus erster Datenexploration 

##### **`twitter_archive`** 
 - Es gibt sowohl Originalbewertungen und Retweets
 - Es müssen Spalten gelöscht werden, die nicht für die Analyse verwendet werden
 - Hundestufen sind nicht korrekt angegeben
 - Zeitstempel ist in einem nicht korrekten Datenformat (hier Datetime)
 - Fehlende Daten in den folgenden Spalten: in_reply_to_status_id, in_reply_to_user_id, retweeted_status_id, retweeted_status_user_id, retweeted_status_timestamp, expanded_urls
 - Der Nenner der Bewertung wird größer als 10
 - Wenn der Zähler eine Dezimalzahl ist wie z.B. 13.5 steht in der Spalte rating_numerator nur die ".5" 
 - Manchmal ist der Name des Hundes None
 - Hundeentwicklungsstufen sind in 4 Spalten
 - Alle drei Quellen sollen zum Schluss in einem DataFrame sein
 
  
 ##### **`tweet_json`**
 - tweet_id ist ein String und keine Zahl
 - fehlende Daten für die tweet_ids
 - Retweets in diesem df
 - Alle drei Quellen sollen zum Schluss in einem DataFrame sein
 
 
 ##### **`image_prediction`** 
 - p1, p2, p3 Hunderassen sind manchmal groß, manchmal klein geschrieben
 - Die Spalten p1, p2 und p3 enthalten ungültige Daten, wie z. B. ein Vogelhaus, einen Dosenöffner oder einen Brustpanzer usw.
 - Doppelte Zeilen
 - Alle drei Quellen sollen zum Schluss in einem DataFrame sein

## Aufgaben für euer Data Cleaning

**`twitter_archive`**</br>
1. Lösche alle Retweets und behalte die Originalbewertungen in `twitter_archive`
2. Lösche folgende Spalten in `twitter_archive`: 'source','in_reply_to_status_id', 'in_reply_to_user_id', 'retweeted_status_id', 'retweeted_status_user_id', 'retweeted_status_timestamp', 'expanded_urls'
3. Konvertiere den Zeitstempel in `twitter_archive` als Datumsformat
4. Korrigiere den Zähler in `twitter_archive`. Wandle es erst als `float` um, und korrigiere die 6 falschen Werte manuell
5. Entferne alle Zeilen in `twitter_archive`, deren Nenner nicht 10 ist
6. Kombinieren alle Spalten der Dog Stages in eine gemeinsame Spalte in `twitter_archive`


**`tweet_json`**</br>
7. Wandle die tweet_id in ein `int` Format in `tweet_json`</br>
8. Lösche alle Retweets und behalte die Originalbewertungen in `tweet_json`
</br></br>
**`image_predictions`**</br>
9. Überabeite die Spalten der Hunderassen in `image_predictions` so, dass nur Kleinbuchstaben enthalten sind</br>
10. Lösche alle duplizierten Zeilen (ca. 66) mit Bild in `image_predictions`
</br></br>
**Übergreifend**</br>
11. Erstelle einen DataFrame aus den drei bereinigten DataFrames mit Namen `df`</br>
12. Nenne die Spalte text in tweet_text um im Dataframe `df` </br>
13. Outlier Rating_numerator erkennen und droppen, mit Hilfe der IQR

In [25]:
'''
Wir erstellen im ersten Schritt immer eine "Hartcopy", 
auf deren Basis wir dann das Data Cleaning machen damit wir später immer Vergleichen können,
ob die Transformationen wirklich funktioniert haben
'''
twitter_archive_clean = twitter_archive.copy()
image_prediction_clean = image_prediction.copy()
tweet_json_clean = tweet_json.copy()

In [26]:
twitter_archive_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   tweet_id                    2356 non-null   int64  
 1   in_reply_to_status_id       78 non-null     float64
 2   in_reply_to_user_id         78 non-null     float64
 3   timestamp                   2356 non-null   object 
 4   source                      2356 non-null   object 
 5   text                        2356 non-null   object 
 6   retweeted_status_id         181 non-null    float64
 7   retweeted_status_user_id    181 non-null    float64
 8   retweeted_status_timestamp  181 non-null    object 
 9   expanded_urls               2297 non-null   object 
 10  rating_numerator            2356 non-null   int64  
 11  rating_denominator          2356 non-null   int64  
 12  name                        2356 non-null   object 
 13  doggo                       2356 

**1. Lösche alle Retweets und behalte die Originalbewertungen**

In [27]:
#CODE
retweets = twitter_archive.loc[twitter_archive['retweeted_status_id'].notnull()].index
twitter_archive_clean = twitter_archive.drop(retweets)

In [28]:
twitter_archive_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2175 entries, 0 to 2355
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   tweet_id                    2175 non-null   int64  
 1   in_reply_to_status_id       78 non-null     float64
 2   in_reply_to_user_id         78 non-null     float64
 3   timestamp                   2175 non-null   object 
 4   source                      2175 non-null   object 
 5   text                        2175 non-null   object 
 6   retweeted_status_id         0 non-null      float64
 7   retweeted_status_user_id    0 non-null      float64
 8   retweeted_status_timestamp  0 non-null      object 
 9   expanded_urls               2117 non-null   object 
 10  rating_numerator            2175 non-null   int64  
 11  rating_denominator          2175 non-null   int64  
 12  name                        2175 non-null   object 
 13  doggo                       2175 

**2. Lösche folgende Spalten in `twitter_archive`: 'source','in_reply_to_status_id', 'in_reply_to_user_id', 'retweeted_status_id', 'retweeted_status_user_id', 'retweeted_status_timestamp', 'expanded_urls'**

In [29]:
#CODE
twitter_archive_clean = twitter_archive.drop(['source','in_reply_to_status_id', 'in_reply_to_user_id', 'retweeted_status_id', 'retweeted_status_user_id', 'retweeted_status_timestamp', 'expanded_urls'], axis=1)

In [30]:
#TEST
twitter_archive_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   tweet_id            2356 non-null   int64 
 1   timestamp           2356 non-null   object
 2   text                2356 non-null   object
 3   rating_numerator    2356 non-null   int64 
 4   rating_denominator  2356 non-null   int64 
 5   name                2356 non-null   object
 6   doggo               2356 non-null   object
 7   floofer             2356 non-null   object
 8   pupper              2356 non-null   object
 9   puppo               2356 non-null   object
dtypes: int64(3), object(7)
memory usage: 184.2+ KB


**3. Konvertiere den Zeitstempel in `twitter_archive` als Datumsformat**

In [31]:
#CODE
twitter_archive_clean["timestamp"] = pd.to_datetime(twitter_archive_clean["timestamp"]).dt.date

In [32]:
#TEST
twitter_archive_clean["timestamp"]

0       2017-08-01
1       2017-08-01
2       2017-07-31
3       2017-07-30
4       2017-07-29
           ...    
2351    2015-11-16
2352    2015-11-16
2353    2015-11-15
2354    2015-11-15
2355    2015-11-15
Name: timestamp, Length: 2356, dtype: object

**4. Korrigiere den Zähler in `twitter_archive`. Wandle es erst als `float` um, und korrigiere die 6 falschen Werte manuell**

In [33]:
#CODE
twitter_archive_clean["rating_numerator"] = twitter_archive_clean["rating_numerator"].astype(float)

#Manuelles Updaten der Nenner (numerators) ????? Zähler????


In [34]:
#TEST
with pd.option_context('max_colwidth', 300):
    display(twitter_archive_clean[twitter_archive_clean['text'].str.contains(r"(\d+\.\d*\/\d+)")]
            [['tweet_id', 'text', 'rating_numerator', 'rating_denominator']])

/var/folders/mq/545q4wvs7qv1ykzg9rcc02380000gn/T/ipykernel_29053/2812561344.py:3: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  display(twitter_archive_clean[twitter_archive_clean['text'].str.contains(r"(\d+\.\d*\/\d+)")]


,tweet_id,text,rating_numerator,rating_denominator
45,883482846933004288,"This is Bella. She hopes her smile made you smile. If not, she is also offering you her favorite monkey. 13.5/10 https://t.co/qjrljjt948",5.0,10
340,832215909146226688,"RT @dog_rates: This is Logan, the Chow who lived. He solemnly swears he's up to lots of good. H*ckin magical af 9.75/10 https://t.co/yBO5wu…",75.0,10
695,786709082849828864,"This is Logan, the Chow who lived. He solemnly swears he's up to lots of good. H*ckin magical af 9.75/10 https://t.co/yBO5wuqaPS",75.0,10
763,778027034220126208,This is Sophie. She's a Jubilant Bush Pupper. Super h*ckin rare. Appears at random just to smile at the locals. 11.27/10 would smile back https://t.co/QFaUiIHxHq,27.0,10
1689,681340665377193984,I've been told there's a slight possibility he's checking his mirror. We'll bump to 9.5/10. Still a menace,5.0,10
1712,680494726643068929,Here we have uncovered an entire battalion of holiday puppers. Average of 11.26/10 https://t.co/eNm2S6p9BD,26.0,10


**5. Entferne alle Zeilen in `tweet_archive`, deren Nenner nicht 10 ist**

In [35]:
f_denominator = len(twitter_archive_clean[twitter_archive_clean["rating_denominator"] != 10])

In [36]:
#CODE
f_denominator = twitter_archive_clean.loc[twitter_archive_clean["rating_denominator"] != 10].index
twitter_archive_clean = twitter_archive_clean.drop(f_denominator)

In [37]:
#TEST
len(twitter_archive_clean[twitter_archive_clean["rating_denominator"] != 10])

0

**6. Kombinieren alle Spalten der Dog Stages in eine gemeinsame Spalte in `twitter_archive`**

In [38]:
#CODE Melt the doggo, floofer, pupper and puppo columns to dogs and dogs_stage column
twitter_archive_clean = pd.melt(twitter_archive_clean, id_vars=['tweet_id',                                          
                                                                'timestamp',
                                                                'text',
                                                                'rating_numerator',
                                                                'rating_denominator',
                                                                'name'],
                               var_name='dogs', value_name='dogs_stage')

In [39]:
twitter_archive_clean.head(10)

,tweet_id,timestamp,text,rating_numerator,rating_denominator,name,dogs,dogs_stage
0,892420643555336193,2017-08-01,This is Phineas. He's a mystical boy. Only eve...,13.0,10,Phineas,doggo,None
1,892177421306343426,2017-08-01,This is Tilly. She's just checking pup on you....,13.0,10,Tilly,doggo,None
2,891815181378084864,2017-07-31,This is Archie. He is a rare Norwegian Pouncin...,12.0,10,Archie,doggo,None
3,891689557279858688,2017-07-30,This is Darla. She commenced a snooze mid meal...,13.0,10,Darla,doggo,None
4,891327558926688256,2017-07-29,This is Franklin. He would like you to stop ca...,12.0,10,Franklin,doggo,None
5,891087950875897856,2017-07-29,Here we have a majestic great white breaching ...,13.0,10,None,doggo,None
6,890971913173991426,2017-07-28,Meet Jax. He enjoys ice cream so much he gets ...,13.0,10,Jax,doggo,None
7,890729181411237888,2017-07-28,When you watch your owner call another dog a g...,13.0,10,None,doggo,None
8,890609185150312448,2017-07-27,This is Zoey. She doesn't want to be one of th...,13.0,10,Zoey,doggo,None
9,890240255349198849,2017-07-26,This is Cassie. She is a college pup. Studying...,14.0,10,Cassie,doggo,doggo


In [40]:
#CODE drop dogs
twitter_archive_clean = twitter_archive_clean.drop("dogs", 1)
#CODE Sort by dogs_stage then drop duplicated based on tweet_id except the last occurrence
twitter_archive_clean = twitter_archive_clean.sort_values(by=["dogs_stage"]).drop_duplicates(subset="tweet_id", keep='last')

/var/folders/mq/545q4wvs7qv1ykzg9rcc02380000gn/T/ipykernel_29053/3601690088.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  twitter_archive_clean = twitter_archive_clean.drop("dogs", 1)


In [41]:
#TEST
twitter_archive_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2333 entries, 2238 to 7658
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   tweet_id            2333 non-null   int64  
 1   timestamp           2333 non-null   object 
 2   text                2333 non-null   object 
 3   rating_numerator    2333 non-null   float64
 4   rating_denominator  2333 non-null   int64  
 5   name                2333 non-null   object 
 6   dogs_stage          2333 non-null   object 
dtypes: float64(1), int64(2), object(4)
memory usage: 145.8+ KB


**7. Wandle die tweet_id in ein `int` Format in `tweet_json`**

In [42]:
#CODE
tweet_json_clean.tweet_id = tweet_json_clean.tweet_id.astype(int)

In [43]:
#TEST
tweet_json_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2331 entries, 0 to 2330
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   tweet_id          2331 non-null   int64  
 1   favorite_count    2331 non-null   int64  
 2   retweet_count     2331 non-null   int64  
 3   followers_count   2331 non-null   int64  
 4   retweeted_status  2331 non-null   object 
 5   url               0 non-null      float64
dtypes: float64(1), int64(4), object(1)
memory usage: 109.4+ KB


**8. Lösche alle Retweets und behalte die Originalbewertungen in `tweet_json`**

In [ ]:
#CODE
tweet_json_clean = tweet_json_clean[tweet_json_clean["retweeted_status"]=="Original tweet"]

In [ ]:
#TEST
tweet_json_clean.retweeted_status.value_counts()

**9. Überarbeite die Spalten der Hunderassen in `image_predictions` so, dass nur Kleinbuchstaben enthalten sind**

In [ ]:
#CODE


In [ ]:
#TEST


**10. Lösche alle duplizierten Zeilen (ca. 66) mit Bild in `image_predictions`**

In [ ]:
#CODE


In [ ]:
#TEST


**11. Erstelle einen DataFrame aus den drei bereinigten DataFrames mit Namen `df`**

In [ ]:
#CODE Erstellen eines neuen Dataframes df, welches twitter_archive_clean und image_prediction_clean merged

#keep rows that have picture (jpg_url)


In [ ]:
#TEST


In [ ]:
#CODE Erstellen eines neuen Dataframes df, welches df_twitter und tweet_json_clean merged


#TEST


**12. Nenne die Spalte text in tweet_text um im Dataframe `df`**

In [ ]:
#CODE


In [ ]:
#TEST


**13. Outlier Rating_numerator erkennen und droppen, mit Hilfe der IQR**

In [ ]:
# CODE - Detect Outlier


In [ ]:
# CODE - Delete Outliers


In [ ]:
# TEST


**FRAGE: Hat es Sinn gemacht, hier die vermeintlichen Outlier rauszuschmeißen?**

# ÜBUNG ENDE
![purple-divider](https://user-images.githubusercontent.com/7065401/52071927-c1cd7100-2562-11e9-908a-dde91ba14e59.png)

## Speichern, Analysieren und Visualisieren der Daten

Ein kleiner Ausblick in die Übung 5, bei der es dann um die Visualisierung von Daten geht.

In [ ]:
#Speichern der twitter_dogs df in eine CSV Datei
df.to_csv('twitter_archive_master.csv', encoding='utf-8', index=False)

In [ ]:
df.info()

In [ ]:
dog_stages = df.groupby("dogs_stage")
dog_stages.describe()

In [ ]:
plt.figure(figsize=(10,10))
g = sns.boxplot(x='dogs_stage',y='retweet_count',data= df,palette='rainbow')
g.axes.set_title('Boxplot dog stages and retweet', fontsize=14);

Doggo stages has huge outliers, there needs to be more investigation to see why this happens at the doggo stages, besides that puppos have on average the most retweets followed by doggo and floofer

In [ ]:
df.plot(y='retweet_count', x='rating_numerator', kind='scatter')
plt.xlabel('Ratings')
plt.ylabel('Retweet Counts')
plt.title('Retweet Counts by Ratings Scatter Plot')

fig = plt.gcf()

As seen here, the better the rating the more retweets happened the most retweets happend between 12-14

In [ ]:
df_dog_type = df.groupby('p1').filter(lambda x: len(x) >= 25)

df_dog_type['p1'].value_counts().plot(kind = 'barh')
plt.title('Histogram of the Most Rated Dog Type')
plt.xlabel('Count')
plt.ylabel('Type of dog')

fig = plt.gcf() 
fig.savefig('output.png',bbox_inches='tight');

In [ ]:
dog_types_count = df_dog_type.p1.value_counts()
dog_types_avg_rating = df_dog_type.groupby('p1').rating_numerator.mean()

df_dog_type.p1.value_counts()

In [ ]:
plt.figure(figsize = (10, 10))
plt.plot(df.timestamp,df.favorite_count, marker = 'o', linestyle = '', ms = 2)
plt.title('Favorite tweet over year')
plt.xlabel('Date')
plt.ylabel('Fovorite Count')
plt.show()

In [ ]:
plt.figure(figsize = (10, 10))
plt.plot(df.timestamp,df.retweet_count, marker = 'o', linestyle = '', ms = 2)
plt.title('Retweets over year')
plt.xlabel('Date')
plt.ylabel('Retweets Count')
plt.show()